In [45]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [46]:
INPUT_SIZE = 3 # thickness, height, angle
LEARNING_RATE = 0.001
BATCH_SIZE = 32 # 
NUM_EPOCHS = 50

TEST_SIZE = 0.2
RANDOM_STATE = 42
device = "cpu"

In [47]:
df = pd.read_csv(r'../MLP/processed_bending_stiffness.csv')

# Remove duplicates
initial_count = len(df)
df = df.drop_duplicates()
removed_count = initial_count - len(df)
if removed_count > 0:
    print(f"Removed {removed_count} duplicate row(s) from the dataset.")
print(f"Shape of dataset after removing duplicates: {df.shape}")
X = df[['Thickness', 'Height', 'Angle (deg)']]
y = df['Bending_Stiffness']

X_train, X_test, y_train, y_test = train_test_split(
    X.values, 
    y.values, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)
DATASET_SIZE = len(df) # Number of samples

Removed 685 duplicate row(s) from the dataset.
Shape of dataset after removing duplicates: (743, 4)


In [48]:
# Make all parameters have mean 0 and std 1 so parameters with a large scale don't skew 
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))  # Add .reshape(-1, 1)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))  # Add .reshape(-1, 1)

# Create DataLoaders
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)  # Now has shape [N, 1]
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

In [49]:
class ConditionalVAE(nn.Module):
    def __init__(self, input_dim=3, condition_dim=1, hidden_dim=64, latent_dim=8, device=device):
        super(ConditionalVAE, self).__init__()

        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2)
            )
        
        # latent space - mean and variance 
        self.mean_layer = nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)

        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, input_dim)
            )
     
    def encode(self, x, condition):
        x_with_condition = torch.cat([x, condition], dim=1)
        x = self.encoder(x_with_condition)
        mean, logvar = self.mean_layer(x), self.logvar_layer(x)
        return mean, logvar

    def reparameterization(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std).to(device)      
        z = mean + std * epsilon
        return z

    def decode(self, z, condition):
        # Concatenate latent with condition
        z = torch.cat([z, condition], dim=1)
        return self.decoder(z)

    def forward(self, x, condition):
        mean, logvar = self.encode(x, condition)
        z = self.reparameterization(mean, logvar)
        x_hat = self.decode(z, condition)
        return x_hat, mean, logvar

def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss + KLD

In [50]:
model = ConditionalVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [51]:
def train(model, optimizer, epochs, device):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        total_samples = 0
        for batch_idx, (x, condition) in enumerate(train_loader):
            x = x.to(device)
            condition = condition.to(device)
            current_batch_size = x.size(0)

            optimizer.zero_grad()

            #Forward pass
            x_hat, mean, log_var = model(x, condition)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            total_samples += current_batch_size
            
            # Backward pass
            loss.backward()
            optimizer.step()

        avg_loss = overall_loss / total_samples
        print("\tEpoch", epoch + 1, "\tAverage Loss: ", (avg_loss))
    return overall_loss

train(model, optimizer, epochs=NUM_EPOCHS, device=device)

	Epoch 1 	Average Loss:  2.929524033157914
	Epoch 2 	Average Loss:  2.3913752302176223
	Epoch 3 	Average Loss:  1.9841505622221565
	Epoch 4 	Average Loss:  1.8233193869542594
	Epoch 5 	Average Loss:  1.6820411104144473
	Epoch 6 	Average Loss:  1.5978994401617082
	Epoch 7 	Average Loss:  1.5586133886266638
	Epoch 8 	Average Loss:  1.4638690659494111
	Epoch 9 	Average Loss:  1.5252472206398293
	Epoch 10 	Average Loss:  1.486820734711207
	Epoch 11 	Average Loss:  1.445204420121832
	Epoch 12 	Average Loss:  1.4652370844625864
	Epoch 13 	Average Loss:  1.4126841098772556
	Epoch 14 	Average Loss:  1.4574933292889836
	Epoch 15 	Average Loss:  1.4028789571640066
	Epoch 16 	Average Loss:  1.3806361638335667
	Epoch 17 	Average Loss:  1.4090166188249684
	Epoch 18 	Average Loss:  1.4694382638642283
	Epoch 19 	Average Loss:  1.403570698567914
	Epoch 20 	Average Loss:  1.4074486915511315
	Epoch 21 	Average Loss:  1.4406371421685524
	Epoch 22 	Average Loss:  1.3790153509840017
	Epoch 23 	Average Loss

810.4686222076416

In [54]:
def generate(model, stiffness, x, y, samples, device='cpu'):
    model.eval()
    with torch.no_grad():
        normalized_stiffness = scaler_y.transform([[stiffness]])
        stiffness_tensor = torch.FloatTensor(normalized_stiffness).to(device) # Normalize 

        condition = stiffness_tensor.repeat(samples, 1) # Repeat for each sample

        z = torch.randn(samples, model.mean_layer.out_features).to(device)

        generated = model.decode(z, condition)

        print(f"Generated: {generated}")

        generated_np = generated.cpu().numpy()
        designs = scaler_X.inverse_transform(generated_np)
    return designs

In [56]:
test_stiffness = 50.0
designs = generate(model, test_stiffness, scaler_X, scaler_y, samples=10, device=device)

print("Generated designs for stiffness", test_stiffness, ":\n", designs)

Generated: tensor([[-0.6357, -1.2914,  0.3160],
        [-0.8601, -1.3972, -0.8480],
        [-0.6426, -1.4498, -0.1624],
        [-0.8935, -1.1694,  0.4484],
        [ 0.3886, -1.5855, -1.6253],
        [-0.0339, -1.4889, -0.7014],
        [-0.2616, -1.5899, -2.0291],
        [-0.7434, -1.4306, -0.6609],
        [ 0.7711, -1.6852, -2.1581],
        [ 1.2103, -1.7387, -2.1039]])
Generated designs for stiffness 50.0 :
 [[ 4.8095355 28.99014   59.367508 ]
 [ 4.473256  25.567125  43.982323 ]
 [ 4.7991514 23.865282  53.044445 ]
 [ 4.423184  32.935165  61.118065 ]
 [ 6.344819  19.47206   33.708008 ]
 [ 5.711499  22.597986  45.92066  ]
 [ 5.370283  19.330229  28.371328 ]
 [ 4.6481566 24.484632  46.455994 ]
 [ 6.917976  16.246134  26.665462 ]
 [ 7.576303  14.517836  27.382212 ]]


In [60]:
target = 50.0
df = pd.read_csv(r'../MLP/processed_bending_stiffness.csv')
df = df.drop_duplicates()
X = df[['Thickness', 'Height', 'Angle (deg)']]
y = df['Bending_Stiffness']
print(df.loc[])


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1400529079.py, line 6)